## 0. Imports

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import countergen

## 1. Choose your dataset
Put your file next to the notebook and give it's filename here:

[Box where you can put your filename, with a convincing default dataset]

[Check that it exists]

[Print a few samples]

In [ ]:
ds = countergen.Dataset.from_jsonl("D:\_Docs\Programmation\Python\Counterfactual-Dataset-Generator\countergen\data\datasets\doublebind.jsonl")

## 2. Choose your augmenters

[Checkboxes with title and explanation]

[Link to somewhere if you want to add your own augmenters]

In [ ]:
augmenters = [countergen.SimpleAugmenter.from_default("gender")]

## 3. Convert!

(Just run this cell)

In [ ]:
aug_ds = countergen.data_augmentation.generate_all_variations(augmenters, ds)

## 4. Choose models to evaluate

[Checkboxs of different models compatible with the dataset you have chosen]

[Link to somewhere if you want to evaluate other models]

In [ ]:
from countergen.utils import unwrap_or, get_device
from countergen.generative_models import get_evaluator_for_model
from transformers import GPT2LMHeadModel
import torch

model_names = ["distilgpt2"]
device = get_device()
model: torch.nn.Module = GPT2LMHeadModel.from_pretrained(model_names[0]).to(device)
model_ev = get_evaluator_for_model(model, "probability")

## 4b. Choose your metrics

[Checkboxs of different metrics & aggregation methods]

[Link to somewhere if you want to add your own]

In [ ]:
aggregator = countergen.aggregators.PerformanceStatsPerCategory()

## 5. Evaluate!

(Just run this cell)

In [ ]:
results = countergen.evaluation.evaluate(aug_ds.samples, model_ev, aggregator)

## 6. The results

[Beautiful displays]

In [ ]:
aggregator.display({"distilgpt2":results})

## 7. Edit...

In [ ]:
from countergen.editing.activation_utils import get_corresponding_activations, get_mlp_modules
layers_dict = get_mlp_modules(model, [2,3])
layers = list(layers_dict.values())
activations = get_corresponding_activations(aug_ds.samples, model, layers)

In [ ]:
from countergen.editing.activation_ds import ActivationsDataset
act_ds = ActivationsDataset.from_activations(activations, device=device)

In [ ]:
from countergen.editing.direction_algos import inlp
dirs = inlp(act_ds, n_training_iters=10)

In [ ]:
from countergen.editing.editor import edit_model, ReplacementConfig
configs = ReplacementConfig.from_module_dict(layers_dict, dirs, has_leftover=False)
new_model = edit_model(model, configs=configs)

In [ ]:
new_model_ev = get_evaluator_for_model(new_model, "probability")
new_results = countergen.evaluation.evaluate(aug_ds.samples, new_model_ev, aggregator)
aggregator.display({"newdistilgpt2":new_results})